# Телеком — задача проекта

Оператору связи «Ниединогоразрыва.ком» необходимо прогнозировать отток клиентов. Если выяснится, что пользователь планирует уйти, ему будут предложены промокоды и специальные условия. Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и договорах.
Наша задача построить модель, которая поможет предсказать уход клиента.

### Вводные данные

Оператор предоставляет два основных типа услуг: 

1. Стационарную телефонную связь. Возможно подключение телефонного аппарата к нескольким линиям одновременно.
2. Интернет. Подключение может быть двух типов: через телефонную линию (DSL*,* от англ. *digital subscriber line*, «цифровая абонентская линия») или оптоволоконный кабель (*Fiber optic*).  

Также доступны такие услуги:

- Интернет-безопасность: антивирус (*DeviceProtection*) и блокировка небезопасных сайтов (*OnlineSecurity*);
- Выделенная линия технической поддержки (*TechSupport*);
- Облачное хранилище файлов для резервного копирования данных (*OnlineBackup*);
- Стриминговое телевидение (*StreamingTV*) и каталог фильмов (*StreamingMovies*).

За услуги клиенты могут платить каждый месяц или заключить договор на 1–2 года. Доступны различные способы расчёта и возможность получения электронного чека.

BeginDate – дата начала пользования услугами,<br>
EndDate – дата окончания пользования услугами,<br>
Type – тип договора: ежемесячный, годовой и т.д,<br>
PaperlessBilling – факт выставления счёта на электронную почту,<br>
PaymentMethod – способ оплаты,<br>
MonthlyCharges – ежемесячные траты на услуги,<br>
TotalCharges – всего потрачено денег на услуги,<br>
Dependents – наличие иждивенцев,<br>
Senior Citizen – наличие пенсионного статуса по возрасту,<br>
Partner – наличие супруга(и),<br>
MultipleLines – наличие возможности ведения параллельных линий во время звонка.<br>

Во всех файлах столбец `customerID` содержит код клиента.

Информация о договорах актуальна на 1 февраля 2020.

### План работы
    1. Подготовка данных. Загрузить и изучить данные. Первичный анализ данных. Вывод.
    2. Анализ данных. 
        2.1 Обработка пропусков и некорректных значений (если есть).
        2.2 Выделение целевого признака. Создание новых признаков, "склейка" датасетов.
    4. Модель. Подготовка выборок для обучения моделей: разделение на выборки, масштабирование, кодироание признаков.
        4.1 Обучение моделей на трейн-выборке, подбор гиперпараметров.
        4.2 Выбор модели, тестовая выборка. 
    5. Общий вывод.

## Подготовка данных.

In [175]:
# импорты сторонних библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# импорты модулей текущего проекта
# пока ничего


from numpy.random import RandomState
state = RandomState(270323) 

In [176]:
def df_info(df: pd.DataFrame):   # функция подсчета пропусков в датафрейме
    df.info()
    print('--------------------------------------')
    print("Количесвто дубликатов:",df.duplicated().sum())
    print("Количество пропусков в таблице :")
    print(f"{df.isna().sum()}")
    print('--------------------------------------')
    display(df)

In [177]:
def readcvs(df: pd.DataFrame):
    try:
        df = pd.read_csv('/datasets/final_provider/'+ df)
    except FileNotFoundError:
        df = pd.read_csv(df)
    return df

In [178]:
df_contract = readcvs('contract.csv') # информация о договоре
df_internet = readcvs('internet.csv') # информация об интернет-услугах
df_personal = readcvs('personal.csv') # персональные данные клиента
df_phone = readcvs('phone.csv') # информация об услугах телефонии

In [179]:
# Смотрим датасет "информация о договоре"
#df_contact

In [180]:
df_info(df_contact)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB
--------------------------------------
Количесвто дубликатов: 0
Количество пропусков в таблице :
customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
dtype: int64
--------------------------------------


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65
...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.5
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.9
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.6


In [181]:
# Смотрим датасет "информация об интернет-услугах"
#df_internet

In [182]:
df_info(df_internet)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB
--------------------------------------
Количесвто дубликатов: 0
Количество пропусков в таблице :
customerID          0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
dtype: int64
--------------------------------------


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...
5512,6840-RESVB,DSL,Yes,No,Yes,Yes,Yes,Yes
5513,2234-XADUH,Fiber optic,No,Yes,Yes,No,Yes,Yes
5514,4801-JZAZL,DSL,Yes,No,No,No,No,No
5515,8361-LTMKD,Fiber optic,No,No,No,No,No,No


In [184]:
# Смотрим датасет "персональные данные клиента"
df_info(df_personal)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB
--------------------------------------
Количесвто дубликатов: 0
Количество пропусков в таблице :
customerID       0
gender           0
SeniorCitizen    0
Partner          0
Dependents       0
dtype: int64
--------------------------------------


,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No
...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes
7039,2234-XADUH,Female,0,Yes,Yes
7040,4801-JZAZL,Female,0,Yes,Yes
7041,8361-LTMKD,Male,1,Yes,No


In [185]:
# Смотрим датасет "информация об услугах телефонии"

In [186]:
df_info(df_phone)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB
--------------------------------------
Количесвто дубликатов: 0
Количество пропусков в таблице :
customerID       0
MultipleLines    0
dtype: int64
--------------------------------------


,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes
...,...,...
6356,2569-WGERO,No
6357,6840-RESVB,Yes
6358,2234-XADUH,Yes
6359,8361-LTMKD,Yes


### Вывод по пункту 1.

В представленных данных отсутствуют пропуски и явные дубликаты.<br> Поля <b>BeginDate</b> и <b>EndDate</b>  имеют тип object  - изменим на дату.<br> Таблицы имею разный размер: <b>df_contact</b> и <b>df_personal</b> содержат по 7043 записей, <b>df_phone</b> - 6361, <b>df_internet</b> - 5517. Связано это с тем что клиенты пользуются разныими видами услуг. В итоговый датасет должно войти 7043 записей. 
        

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b><br> 
Не скажу что все ясно, но пока - НЕТ ВОПРОСОВ.
</div>